In [13]:
import os
import numpy as np
import lxml
from lxml import objectify, etree, html
import json
import urllib3
from io import StringIO, BytesIO
import newspaper
from newspaper import Article
import pandas as pd
import progressbar
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
links = {"AUTO" : "https://www.nn.ru/news/archive/auto/articles/",
         "WINTER" : "https://www.nn.ru/news/archive/winter/articles/",
         "WORLD" : "https://www.nn.ru/news/archive/world/articles/",
         "TOWN" : "https://www.nn.ru/news/archive/gorod/articles/",
         "ACCIDENTS" : "https://www.nn.ru/news/archive/incidents/articles/",
         "PRESSRELEASE" : "https://www.nn.ru/news/archive/pressrelease/articles/"}

In [5]:
doc_links = {}
doc_http = ""
for key in links.keys():
    driver = webdriver.Chrome(executable_path='C:/simple/chromedriver.exe')
    driver.get(links[key])
    elem = driver.find_element_by_xpath("//*[@class='rn-arhive__more-content-link js-rn-arhive__more-content']")
    driver.execute_script("window.scrollTo(0, 480)") 
    try:
        elem_block = driver.find_element_by_xpath("//*[@class='top-fixed-banner__close js-top-fixed-banner-close']")
        sleep(1)
        elem_block.click()
        sleep(1)
    except:
        pass
    for i in range(20):
        try:
            sleep(1)
            elem.click()
            ActionChains(driver).move_to_element(elem).perform()
            sleep(2)
        except:
            break
    doc_http = lxml.html.document_fromstring(driver.page_source)
    doc_links[key] = doc_http.xpath("//ul[@class='rn-arhive__list _rn-info__list__arhive']/li[@class='rn-arhive__item']/div[@class='rn-arhive__item-content']/*[1]/@href")

In [6]:
for key in doc_links.keys():
    print(key + " - " + str(len(doc_links[key])))

AUTO - 83
WINTER - 73
WORLD - 151
TOWN - 420
ACCIDENTS - 105
PRESSRELEASE - 40


In [7]:
import xml.etree.ElementTree as xml
 
def createXML(filename):
    
    root = xml.Element("Catalog")
    theme = xml.Element("Article")
    root.append(theme)
     
    name_of_article = xml.SubElement(theme, "name_of_article")
    name_of_article.text = "test"
    
    author = xml.SubElement(theme, "author")
    author.text = "test"
    
    publish_date = xml.SubElement(theme, "publish_date")
    publish_date.text = "test"
    
    genre = xml.SubElement(theme, "genre")
    
    text = xml.SubElement(theme, "text")
    
    tree = xml.ElementTree(root)
    
    with open(filename, "wb") as fh:
        tree.write(fh)

In [8]:
#Выполнять только 1 раз!
createXML("article.xml")

In [9]:
import time
import xml.etree.cElementTree as ET
 
def editXML(filename, link, ge):
    
    tree = ET.ElementTree(file=filename)
    root = tree.getroot()
    ar = xml.Element("Article")
    root.append(ar)
    
    try:
        
        article = Article(link)
        article.download()
        article.parse()
        
        name_of_article = xml.SubElement(ar, "name_of_article")
        name_of_article.text = article.title

        author = xml.SubElement(ar, "author")
        author.text = article.authors[0]

        publish_date = xml.SubElement(ar, "publish_date")
        publish_date.text = str(article.publish_date)

        genre = xml.SubElement(ar, "genre")
        genre.text = ge

        #Очистка данных(simply)
        txt = xml.SubElement(ar, "text")
        clear_text = article.text

        err = '\n'
        for i in err:
            clear_text = clear_text.replace(i, " ")

        txt.text = ' '.join(clear_text.split())
        
    except:
        pass
    
    tree = ET.ElementTree(root)
    
    with open("updated.xml", "wb") as f:
        tree.write(f)

In [12]:
editXML("article.xml", doc_links["AUTO"][0], "AUTO")
for key in doc_links.keys():
    for item in progressbar.progressbar(doc_links[key]):
        editXML("updated.xml", item, key)

100% (83 of 83) |########################| Elapsed Time: 0:02:08 Time:  0:02:08
100% (73 of 73) |########################| Elapsed Time: 0:02:07 Time:  0:02:07
100% (151 of 151) |######################| Elapsed Time: 0:04:33 Time:  0:04:33
100% (420 of 420) |######################| Elapsed Time: 0:18:10 Time:  0:18:10
100% (105 of 105) |######################| Elapsed Time: 0:05:24 Time:  0:05:24
100% (40 of 40) |########################| Elapsed Time: 0:02:07 Time:  0:02:07


In [10]:
editXML("article.xml", doc_links["AUTO"][0], "AUTO")

In [11]:
article = Article(link)
article.download()
article.parse()
clear_text = article.text

err = '\n'
for i in err:
    clear_text = test.replace(i, " ")

' '.join(clear_text.split())

NameError: name 'link' is not defined